# SurvHive test

Let's compare the performance of three methods (Cox-based, tree-based and deep-learning-based) on the same dataset.
First the default parameters will be evaluated then a small random search on a grid will be performed.

In [7]:
import numpy
import sklearn
import survhive
import pandas
from sksurv.datasets import get_x_y

In [2]:
seed=202405

Get data from CSV

In [18]:
data_df=pandas.read_csv('survhive/datasets/metabric.csv')
data_df

,Unnamed: 0,hormone_treatment,radiotherapy,chemoterapy,ER-positive,MKI67,EGFR,PGR,ERBB2,AOD,time,event
0,0,5.603834,7.811392,10.797988,5.967607,1.0,1.0,0.0,1.0,56.840000,99.333336,0
1,1,5.284882,9.581043,10.204620,5.664970,1.0,0.0,0.0,1.0,85.940002,95.733330,1
2,2,5.920251,6.776564,12.431715,5.873857,0.0,1.0,0.0,1.0,48.439999,140.233337,0
3,3,6.654017,5.341846,8.646379,5.655888,0.0,0.0,0.0,0.0,66.910004,239.300003,0
4,4,5.456747,5.339741,10.555724,6.008429,1.0,0.0,0.0,1.0,67.849998,56.933334,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1898,1898,5.946987,5.370492,12.345780,5.741395,1.0,1.0,0.0,1.0,76.839996,87.233330,1
1899,1899,5.339228,5.408853,12.176101,5.693043,1.0,1.0,0.0,1.0,63.090000,157.533340,0
1900,1900,5.901610,5.272237,14.200950,6.139390,0.0,0.0,0.0,1.0,57.770000,37.866665,1
1901,1901,6.818109,5.372744,11.652624,6.077852,1.0,0.0,0.0,1.0,58.889999,198.433334,0


In [19]:
data_df.drop('Unnamed: 0',axis=1,inplace=True)

convert data to sksurv format

In [20]:
X, y = get_x_y(data_df,attr_labels=['event','time'], pos_label=True)
X, y

(      hormone_treatment  radiotherapy  chemoterapy  ER-positive  MKI67  EGFR  \
 0              5.603834      7.811392    10.797988     5.967607    1.0   1.0   
 1              5.284882      9.581043    10.204620     5.664970    1.0   0.0   
 2              5.920251      6.776564    12.431715     5.873857    0.0   1.0   
 3              6.654017      5.341846     8.646379     5.655888    0.0   0.0   
 4              5.456747      5.339741    10.555724     6.008429    1.0   0.0   
 ...                 ...           ...          ...          ...    ...   ...   
 1898           5.946987      5.370492    12.345780     5.741395    1.0   1.0   
 1899           5.339228      5.408853    12.176101     5.693043    1.0   1.0   
 1900           5.901610      5.272237    14.200950     6.139390    0.0   0.0   
 1901           6.818109      5.372744    11.652624     6.077852    1.0   0.0   
 1902           5.725708      5.449718     9.680736     6.595955    1.0   1.0   
 
       PGR  ERBB2        A

get data in scikit-survival format

In [21]:
#X, y = data_ds.get_X_y()
X.shape, y.shape

((1903, 9), (1903,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
X_train, X_test, y_train, y_test = survhive.survival_train_test_split(X, y, rng_seed=seed)

In [24]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((1427, 9), (476, 9))

In [25]:
survhive.get_indicator(y).sum(), survhive.get_indicator(y_train).sum(), survhive.get_indicator(y_test).sum(),


(1103, 827, 276)

Set-up a dict to collect test results

In [26]:
vanilla_mods= {}
antolini_vanilla_test = {}

In [27]:
def model_class(_model):
    return type(_model).__name__

## setup a few vanilla models

Coxnet and GradientBoostingSA from sksurv, survtracesingle from survtrace 

In [28]:
vanilla_mods['CoxNet'] =survhive.CoxNet(rng_seed=seed)
vanilla_mods['GrBoostSA'] =survhive.GrBoostSA(rng_seed=seed)
vanilla_mods['SurvTraceSingle'] =survhive.SurvTraceSingle(rng_seed=seed)

In [29]:
for _ in vanilla_mods.keys():
    vanilla_mods[_].fit(X_train, y_train)
    antolini_vanilla_test[_] = vanilla_mods[_].score(X_test, y_test)

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")


use pytorch-cuda for training.


/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/survtrace/train_utils.py:208: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1706629241544/work/torch/csrc/utils/python_arg_parser.cpp:1519.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


[Train-0]: 34.084216475486755
[Val-0]: 1.6006633043289185
[Train-1]: 25.84661716222763
[Val-1]: 1.13923180103302
[Train-2]: 22.774731874465942
[Val-2]: 1.2873245477676392
EarlyStopping counter: 1 out of 5
[Train-3]: 23.48691302537918
[Val-3]: 1.1479052305221558
EarlyStopping counter: 2 out of 5
[Train-4]: 22.957712769508362
[Val-4]: 1.1559746265411377
EarlyStopping counter: 3 out of 5
[Train-5]: 22.733501732349396
[Val-5]: 1.1555413007736206
EarlyStopping counter: 4 out of 5
[Train-6]: 22.73248988389969
[Val-6]: 1.1325762271881104
[Train-7]: 23.14801973104477
[Val-7]: 1.1346057653427124
EarlyStopping counter: 1 out of 5
[Train-8]: 22.605539560317993
[Val-8]: 1.1324126720428467
[Train-9]: 22.75981366634369
[Val-9]: 1.123903512954712
[Train-10]: 22.489030838012695
[Val-10]: 1.1525746583938599
EarlyStopping counter: 1 out of 5
[Train-11]: 23.76988512277603
[Val-11]: 1.137022852897644
EarlyStopping counter: 2 out of 5
[Train-12]: 22.280696272850037
[Val-12]: 1.1425774097442627
EarlyStoppin

In [30]:
antolini_vanilla_test

{'CoxNet': 0.6230663234797078,
 'GrBoostSA': 0.6188414840236071,
 'SurvTraceSingle': 0.6338299144632504}

In [31]:
vanilla_df = pandas.DataFrame(data=antolini_vanilla_test.values(),
                              index=antolini_vanilla_test.keys(),
                             columns=['Defaults'])
vanilla_df

,Defaults
CoxNet,0.623066
GrBoostSA,0.618841
SurvTraceSingle,0.633830


# Parameter optimization

In [32]:
# max number of processes (CPU)
cpus=4
antolini_best_test = {}
from IPython.display import display
#display(pd1)
#display(pd2)

## Optimization loop

Now let's perform some random search on the model parameter space.

Each optimization step does a two repeats of five-fold cross validations. 
The score is the average on the internal cross validated tests.

In [33]:
#vanilla_mods['CoxNet'].get_parameter_grid()
for _model in vanilla_mods.keys():
    print("Optimizing", _model)
    # perform some points of random search on default search grid
    _opt_model, _opt_model_params, _opt_model_search = survhive.optimize(vanilla_mods[_model], 
                                                                         X_train, y_train, 
                                                                         mode='sklearn-random', tries=10, 
                                                                         n_jobs=cpus)
    print("Top-ranking model for", _model)
    print('N.B. mean_test_score is the mean of the cross-validation runs on the *training* data')
    display(survhive.get_model_top_ranking_df(_opt_model_search))
    print("Top-ten models for", _model)
    display(survhive.get_model_scores_df(_opt_model_search)[:10])
    antolini_best_test[_model]=_opt_model.score(X_test,y_test)

antolini_best_test

Optimizing CoxNet
Random search tries: 10
Top-ranking model for CoxNet
N.B. mean_test_score is the mean of the cross-validation runs on the *training* data


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
8,1,0.636914,0.018524,"{'l1_ratio': 0.01, 'alpha': 0.001}",0.005878,0.001409


Top-ten models for CoxNet


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
8,1,0.636914,0.018524,"{'l1_ratio': 0.01, 'alpha': 0.001}",0.005878,0.001409
2,2,0.636889,0.018477,"{'l1_ratio': 0.01, 'alpha': 0.005}",0.00612,0.001425
5,3,0.636744,0.017737,"{'l1_ratio': 0.1, 'alpha': 0.06}",0.005951,0.001484
9,4,0.636515,0.018635,"{'l1_ratio': 0.75, 'alpha': 0.005}",0.006229,0.00179
3,5,0.636469,0.018595,"{'l1_ratio': 0.9, 'alpha': 0.005}",0.005908,0.001276
4,6,0.636432,0.018501,"{'l1_ratio': 0.9, 'alpha': 0.008}",0.005927,0.001429
7,7,0.634473,0.017474,"{'l1_ratio': 0.25, 'alpha': 0.07}",0.005823,0.001377
0,8,0.622717,0.020459,"{'l1_ratio': 0.75, 'alpha': 0.07}",0.005479,0.000219
1,9,0.600229,0.019333,"{'l1_ratio': 0.25, 'alpha': 0.3}",0.005155,0.000104
6,10,0.539348,0.041217,"{'l1_ratio': 0.75, 'alpha': 0.3}",0.005287,0.0003


Optimizing GrBoostSA
Random search tries: 10
Top-ranking model for GrBoostSA
N.B. mean_test_score is the mean of the cross-validation runs on the *training* data


...

use pytorch-cuda for training.
[Train-0]: 34.76032865047455
[Val-0]: 1.441606044769287
[Train-1]: 28.724479913711548
[Val-1]: 1.2014213800430298
[Train-2]: 23.498599767684937
[Val-2]: 1.31163489818573
EarlyStopping counter: 1 out of 5
[Train-3]: 23.495606780052185
[Val-3]: 1.1395959854125977
[Train-4]: 23.026306569576263
[Val-4]: 1.1427233219146729
EarlyStopping counter: 1 out of 5
[Train-5]: 22.86152881383896
[Val-5]: 1.1506757736206055
EarlyStopping counter: 2 out of 5
[Train-6]: 22.985354602336884
[Val-6]: 1.126690149307251
[Train-7]: 23.207220375537872
[Val-7]: 1.148396611213684
EarlyStopping counter: 1 out of 5
[Train-8]: 22.721343398094177
[Val-8]: 1.1280063390731812
EarlyStopping counter: 2 out of 5
[Train-9]: 22.806435644626617
[Val-9]: 1.1199284791946411
[Train-10]: 22.638912975788116
[Val-10]: 1.1424498558044434
EarlyStopping counter: 1 out of 5
[Train-11]: 23.80429458618164
[Val-11]: 1.125131368637085
EarlyStopping counter: 2 out of 5
[Train-12]: 22.319191992282867
[Val-12]:

,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
9,1,0.668,0.022161,"{'num_hidden_layers': 4, 'num_attention_heads'...",10.751444,4.662867


Top-ten models for SurvTraceSingle


,rank_test_score,mean_test_score,std_test_score,params,mean_fit_time,std_fit_time
9,1,0.668,0.022161,"{'num_hidden_layers': 4, 'num_attention_heads'...",10.751444,4.662867
1,2,0.667083,0.025004,"{'num_hidden_layers': 4, 'num_attention_heads'...",11.306713,2.171977
5,3,0.666672,0.024035,"{'num_hidden_layers': 3, 'num_attention_heads'...",8.343928,1.893773
6,4,0.666233,0.020647,"{'num_hidden_layers': 2, 'num_attention_heads'...",6.403396,1.371994
2,5,0.66623,0.025355,"{'num_hidden_layers': 2, 'num_attention_heads'...",5.75527,1.591383
3,6,0.665892,0.020556,"{'num_hidden_layers': 2, 'num_attention_heads'...",6.892089,1.944336
7,7,0.665839,0.026565,"{'num_hidden_layers': 2, 'num_attention_heads'...",7.092211,2.330553
8,8,0.66538,0.019404,"{'num_hidden_layers': 3, 'num_attention_heads'...",7.708863,2.315464
4,9,0.663828,0.020076,"{'num_hidden_layers': 4, 'num_attention_heads'...",10.88873,2.399082
0,10,0.66062,0.029758,"{'num_hidden_layers': 4, 'num_attention_heads'...",10.545536,3.144926


{'CoxNet': 0.6232743155760081,
 'GrBoostSA': 0.6154746119647453,
 'SurvTraceSingle': 0.6290070977302863}

# Final results

Performace of models expressed as an Antolini score on the test set.

In [34]:
optimized_df = pandas.DataFrame(data=antolini_best_test.values(),
                              index=antolini_best_test.keys(),
                             columns=['Somewhat optimized'])

In [35]:
pandas.concat([vanilla_df, optimized_df],axis=1)

,Defaults,Somewhat optimized
CoxNet,0.623066,0.623274
GrBoostSA,0.618841,0.615475
SurvTraceSingle,0.633830,0.629007
